# 過去スレ保管庫よりリンクをスクレイピング

In [1]:
#データ管理のID
ID = "mimikkyu"

#リンク保存するファイル
links_file = "Links/%s.txt"%ID

#過去スレ一覧のURL（一つでいい）
first_url = "https://medaka.5ch.net/poke/kako/kako0000.html"

## スクレイピング

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_binary
from time import sleep
import concurrent.futures

### ブラウザの用意

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options = chrome_options)

### 各サーバーのリンクを取得

In [4]:
server_urls = [first_url]

driver.get(first_url)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

#左のメニューより取得
menu_div = soup.find("div", class_ = "menu")

server_list = menu_div.find("h2")

for p in menu_div.find_all("p", class_="menu_link"):
    a = p.find("a")
    #「このサーバー」ではない
    if a != None: 
        if a.get("href")[0] != ".":
            #Other Listではない
            link = "https:" + a.get("href")
            server_urls.append(link)

server_urls[:5], len(server_urls)

(['https://medaka.5ch.net/poke/kako/kako0000.html',
  'https://game.5ch.net/poke/kako/',
  'https://game10.5ch.net/poke/kako/',
  'https://game11.5ch.net/poke/kako/',
  'https://game13.5ch.net/poke/kako/'],
 15)

### サーバー内のリスト一覧を取得

In [5]:
list_urls = []

#各サーバーについて
for server in server_urls:
    #自分自身
    list_urls.append(server)

    driver.get(server)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    #左のメニューより取得
    menu_div = soup.find("div", class_ = "menu")

    server_list = menu_div.find("h2")

    for p in menu_div.find_all("p", class_="menu_link"):
        a = p.find("a")
        #「このサーバー」ではない
        if a != None: 
            if a.get("href")[0] == ".":
                #Other Listである
                link = server + a.get("href")[2:]
                list_urls.append(link)
            else:
                break
    
list_urls[:2], server_urls[:2]

(['https://medaka.5ch.net/poke/kako/kako0000.html',
  'https://game.5ch.net/poke/kako/'],
 ['https://medaka.5ch.net/poke/kako/kako0000.html',
  'https://game.5ch.net/poke/kako/'])

### 各リストから、URLを取得

In [6]:
def scrape(list_url):
    driver.get(list_url)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    output = []

    for p in soup.find_all("p", class_ = "main_odd"):
        href = p.find("a").get("href")

        #URLを組み立てる
        link = "https://" + list_url.split("/")[2] +href

        output.append(link) 

    for p in soup.find_all("p", class_ = "main_even"):
        href = p.find("a").get("href")

        #URLを組み立てる
        link = "https://" + list_url.split("/")[2] +href

        output.append(link) 

    return output 


result_urls = []
for list_url in list_urls:
    result_urls.extend(scrape(list_url))

result_urls[:5], len(result_urls)    

(['https://medaka.5ch.net/test/read.cgi/poke/1676083464/',
  'https://medaka.5ch.net/test/read.cgi/poke/1676036068/',
  'https://medaka.5ch.net/test/read.cgi/poke/1675860039/',
  'https://medaka.5ch.net/test/read.cgi/poke/1675721015/',
  'https://medaka.5ch.net/test/read.cgi/poke/1675614440/'],
 68776)

## 結果を保存

In [14]:
with open(links_file, "a", encoding = "utf-8") as f:
    for link in result_urls:
        f.write(link + "\n")